In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
import yfinance as yf

In [2]:
class DataBase:
    def __init__(self, kind):
        self.kind = kind
        self.data = {}
        if kind == 'crypto':
            self.add = self._add_crypto
        elif kind == 'stock':
            self.add = self._add_stock
        else:
            raise ValueError(f'Found no kind named: {kind}')
    
    def _add_crypto(self, name, color, symbol, amount, category, yf_add_ons='-USD'):
        if name not in self.data:
            try:
                self.data[name] = {
                    'color': color,
                    'symbol': symbol,
                    'current_price': yf.Ticker(f'{symbol}{yf_add_ons}').history(period='1d')['Open'][0],
                    'start_date': datetime.datetime.strptime(yf.Ticker(f'{symbol}{yf_add_ons}').info['startDate'], '%Y-%m-%d'),
                    'portfolio_amount': amount, 
                    'kind': 'crypto',
                    'category': category,
                    'symbol_add_on': yf_add_ons
                }
            except Exception as e:
                print(f'An {e} has occurred while extracting data for: {name}')
        else:
            print(f'{name} already in DataBase')
        
    def _add_stock(self, name, color, symbol, amount):
        if name not in self.data:
            try:
                self.data[name] = {
                    'color': color,
                    'symbol': symbol,
                    'current_price': yf.Ticker(symbol).history(period='1d')['Open'][0],
                    'start_date': datetime.datetime.strptime(yf.Ticker(symbol).info['startDate'], '%Y-%m-%d'),
                    'portfolio_amount': amount, 
                    'kind': 'stock'
                }
            except Exception as e:
                print(f'An {e} has occurred while extracting data for: {name}')
        else:
            print(f'{name} already in DataBase')
        
    def remove(self, name):
        if name in self.data:
            del self.data[name]
        else:
            print(f'{name} not found in DataBase')
    
    def get_info(self, name):
        return self.data.get(name, f'{name} not found in DataBase')
    
    def update_amount(self, name, amount):
        if name in self.data:
            self.data[name]['portfolio_amount'] = amount
        else:
            print(f'{name} not found in DataBase')
        
    def update_prices(self):
        if self.kind == 'crypto':
            for name, details in self.data.items():
                self.data[name] = yf.Ticker(details['symbol']+details['symbol_add_on']).history(period='1d')['Open'][0]
        elif self.kind =='stock':
            for name, details in self.data.items():
                self.data[name] = yf.Ticker(details['symbol']).history(period='1d')['Open'][0]
    
    def filter_by_category(self, categories):
        filterd = {}
        for name, details in self.data.items():
            if details['category'] in categories:
                filterd[name] = details
        return filterd 
    
    def calculate_distribution(self, total_investment, distribution):
        for category_name, category_info in distribution.items():
            category_total = total_investment * (category_info['precentage'] / 100)
            for asset_name, precentage in category_info['assets'].items():
                amount_in_currency = category_total * (precentage / 100)
                if asset_name in self.data:
                    self.data[asset_name]['portfolio_amount'] = amount_in_currency / self.data[asset_name]['current_price']
                else:
                    print(f"{asset} is not in the DataBase.")
    
    @staticmethod
    def create_distribution(*categories):
        distribution = {}
        for category in categories:
            name, precentage, *assets = category
            assets_distribution = {asset_name: asset_percentage for asset_name, asset_percentage in assets}
            distribution[name] = {'precentage': precentage, 'assets': assets_distribution}
        return distribution

In [ ]:
bitcoin = yf.Ticker('BTC-USD')

In [ ]:
db = DataBase('crypto')
db.add('Bitcoin', '#F7931A', 'BTC', 0, 'Foundational Assets')
db.add('Ethereum', '#627EEA', 'ETH', 0, 'Foundational Assets')
db.add('Chainlink', '#0040DE', 'LINK', 0, 'Interoperability Protocols')

In [ ]:
assets_dict = db.data
start_date_string = pd.DataFrame(assets_dict).T['start_date'].max().date().strftime('%Y-%m-%d')
start_date = datetime.datetime.strptime(start_date_string, '%Y-%m-%d').date()

In [ ]:
assets_dict

In [ ]:
# Making the price history DataFrame
my_assets = [asset for asset in assets_dict]
price_history = pd.DataFrame()

for asset in my_assets:
    
    ticker = assets_dict[asset]['symbol']+assets_dict[asset]['symbol_add_on']
    
    try:
        Open = yf.Ticker(ticker).history(start=start_date_string)['Open']
        
    except Exception as e:
                print(f'An {e} has occurred while extracting data for: {asset}')
            
    Open.index = pd.DatetimeIndex(Open.index.date)
    
    price_history[asset] = Open

price_history.fillna(method='ffill', inplace=True)
price_history.fillna(method='backfill', inplace=True)

In [ ]:
# Different types of returns
def arithmetic_returns(price):
    return (price/price.shift(1)) - 1

def logarithmic_returns(price):
    return np.log(price/price.shift(1))

# making the return history DataFrame
return_history = pd.DataFrame(columns=[f'{asset} return' for asset in price_history.columns])

for asset in price_history.columns:
    return_history[f'{asset} return'] = logarithmic_returns(price_history[asset]).dropna()
return_history.index = pd.DatetimeIndex(return_history.index.date)